In [ ]:
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
sns.set_style('white')
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARMA
from pandas.plotting import register_matplotlib_converters
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import pmdarima
import arch

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import pmdarima
import arch


year = 2019
ticker = 'IFM'
string = str(year)+'/'+ticker+'.csv'
col_names=['TIME', 'X', 'Y', 'Z'] 

df = pd.read_csv(os.path.join('E:\CTA quant/ml_cta-master/data/index/',string), index_col=0, encoding='gbk')
df.columns = ['code', 'time', 'open', 'high', 'low', 'close', 'volume', 'turnover', 'open interest']

df.rename_axis("type", axis='index', inplace=True)
df['time'] = pd.to_datetime(df['time'])

df.reset_index(inplace=True)

#df = df.iloc[0:100, :]
#df.head(1000)
df['day'] = df['time'].map(lambda x: x.year)*10000 + df['time'].map(lambda x: x.month)*100 + df['time'].map(lambda x: x.day)




def Get_indices(x):
    
    # return during the hold period

    ret2 = sum(x.return_min)
    vol = x.return_min.std(ddof = 0)
    #vol = np.std(x.return_min.std)
    
    return pd.Series([vol,ret2], index=['Volatility','Logreturn2'])

def get_best_model(df):
    best_aic = np.inf 
    best_order = None
    best_mdl = None
    pq_rng = range(8) # [0,1,2,3,4]
    
    for i in pq_rng:
        for j in pq_rng:
            try:
                tmp_mdl = ARMA(df, order=(i,j)).fit(
                    method='mle', trend='nc'
                )
                tmp_aic = tmp_mdl.aic
                if tmp_aic < best_aic:
                    best_aic = tmp_aic
                    best_order = (i, j)
                    best_mdl = tmp_mdl
            except: continue
    print('aic: {:6.2f} | order: {}'.format(best_aic, best_order))                    
    return best_aic, best_order, best_mdl

def plot_corrs(n):
    n = n # the time interval to partition the price series into
    def func(i):
        return (i - i%n)

    df['ind'] = df.index
    df['ind'] = df.ind.apply(func)
    df['return_min'] = (df['close'])/(df['open'])
    df['return_min'] = np.log(df['return_min'])
    #df['return'] = (df['close']-df['open'].shift(periods = n-1))/(df['open'].shift(periods = m-1))
    df1 = df.copy()
    df1 = df.groupby(df.ind)
    thres_up = 0.00
    thres_down= 0.00
    df1 = df1.apply(Get_indices)
#     fig, axes = plt.subplots(2,2, figsize = (10,10))
#     plot_acf(df1.Logreturn2, ax = axes[0,0])
#     plot_acf(df1.Volatility, ax = axes[0,1])
#     plot_pacf(df1.Logreturn2, ax = axes[1,0])
#     plot_pacf(df1.Volatility, ax = axes[1,1])
    
    return df1

def fit_arma(df):
    # find best order
    # automatically fit the optimal ARIMA model for given time series
    # train_len = int(train_test_split * len(df))
    train_len = len(df)
    arima_model_fitted = pmdarima.auto_arima(df[:train_len], start_p = 2, start_q = 2)
    order = arima_model_fitted.get_params()['order']
    # pmdarima.arima.ARIMA
    # predict = arima_model_fitted.predict(n_periods = len(df)-train_len)
    # res = df[train_len:] - predict
    # plot_acf(res)
    arima_residuals = arima_model_fitted.arima_res_.resid
    # the following plots point to the best garch order
    plot_acf(arima_residuals**2)
    plot_pacf(arima_residuals**2)
    return arima_model_fitted, order

def fit_arma_garch(df, roll_window_len, order):
    # df: log return series, roll_window_len: the time window to conduct ARMA-GARCH fitting on, order: order of GARCH
    n_period = 1
    T = roll_window_len
    preds = []
    for d in range(len(df)-T):
        # fit ARMA on returns 
        best_aic, best_order, best_mdl = get_best_model(df[d:d+T])
        #arima_model_fitted = pmdarima.auto_arima(df[d:d+T])
        #p, d, q = arima_model_fitted.order
        arima_residuals = best_mdl.resid

        # fit a GARCH(order) model on the residuals of the ARMA model
        garch = arch.arch_model(arima_residuals, p=order[0], q=order[1])
        garch_fitted = garch.fit()

        # Use ARMA to predict mu
        predicted_mu = best_mdl.forecast()[0]
        # Use GARCH to predict the residual
        garch_forecast = garch_fitted.forecast(horizon=n_period)
        predicted_et = garch_forecast.mean['h.1'].iloc[-1]
        # # Combine both models' output: yt = mu + et
        prediction = predicted_mu + predicted_et
        preds.append(prediction)
    
    
    return preds

def do_everything(n, roll_window_len):
    df1 = plot_corrs(n = n)
    preds = fit_arma_garch(df1['Logreturn2'], roll_window_len = roll_window_len, order = (1,1))
    T = np.arange(len(preds))
    true_val = df1['Logreturn2'][-len(T):]
    plt.plot(T, preds, label = 'prediction')
    plt.plot(T, true_val, label = 'true value')
    plt.title('prediction of return over period of ' + str(n) + ' mins')
    plt.legend()
    
    pred_bool = [int(preds[i] > 0) for i in range(len(preds))]
    true_bool = [int(list(true_val)[i] > 0) for i in range(len(preds))]
    err = sum(abs(np.asarray(true_bool) - np.asarray(pred_bool)))/len(preds)
    accuracy = 1 - err
    return preds, accuracy
    

In [ ]:
preds, accuracy = do_everything(n = 120, roll_window_len=444)

In [ ]:
accuracy